## Section 1

### Project: Generate Parallel Databases

In [1]:
import torch

In [2]:
def create_random_db(num_entries):

    db = torch.rand(num_entries) > 0.5
    return db

def get_parallel_db(db, remove_idx):

    return torch.cat((db[:remove_idx], db[remove_idx+1:]))

def create_parallel_dbs(n_entries):
    all_dbs = []
    db = create_random_db(n_entries)

    for idx in range(len(db)):
        parallel_db = get_parallel_db(db, idx)
        all_dbs.append(parallel_db)

    return db, all_dbs

In [3]:
create_parallel_dbs(5)

(tensor([False,  True, False,  True, False]),
 [tensor([ True, False,  True, False]),
  tensor([False, False,  True, False]),
  tensor([False,  True,  True, False]),
  tensor([False,  True, False, False]),
  tensor([False,  True, False,  True])])

---

## Section 2

### Project: Evaluating the Privacy of a Function

In [4]:
def sensitivity(query, n_entries=2000):

    db, pdbs = create_parallel_dbs(n_entries)
    full_db_res = query(db)

    max_dist = 0
    for pdb in pdbs:
        pdb_res = query(pdb)

        db_dist = torch.abs(pdb_res - full_db_res)

        if (db_dist > max_dist):
            max_dist = db_dist

    return max_dist

In [5]:
def query(db):
    return db.float().mean()

In [6]:
sensitivity(query)

tensor(0.0003)

### Calculate L1 Sensitivity for Threshold

In [7]:
def query(db, threshold=5):
    return (db.sum() > threshold).float()

In [8]:
for i in range(10):
    sensitivity_val = sensitivity(query, 10)
    print(sensitivity_val)

0
0
0
0
0
0
0
0
tensor(1.)
0


### Project: Perform a Differencing Attack

In [9]:
db, _ = create_parallel_dbs(100)

In [10]:
pdb = get_parallel_db(db, remove_idx=10)

In [11]:
# Differencing attacking using sum

sum(db) - sum(pdb)

tensor(1)

In [12]:
# Differencing attacking using mean

(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(0.0063)

In [13]:
# Differencing attacking using threshold

threshold = 49
(sum(db).float() > threshold).int() - (sum(pdb).float() > threshold).int()

tensor(0, dtype=torch.int32)